In [7]:
from gurobipy import *

In [8]:
def branch_and_bound(model, vars, subproblems, Node_number):
    
    Incumbent_obj_val = 0
    best_obj_val = 0  
    
    while subproblems:
        
        # Pick the subproblem with the highest lower bound (Going to another Node)
        subproblem = subproblems.pop(0)
        Node_number = Node_number + 1
        
        # Solve the subproblem
        subproblem.optimize()
        
        if subproblem.status != GRB.OPTIMAL:
            print('Node number', Node_number)
            print('Pruned by infeasibility') 
        
        # If feasible, if integer, set best solution (?????????I have a question here (how to write if for check integer)????????)
        elif subproblem.getObjective().getValue() == 1.180000000e+01:
                
            Incumbent_obj_val = subproblem.getObjective().getValue()
            Incumbent_solution = [var.X for var in subproblem.getVars()]
            best_obj_val = Incumbent_obj_val
            best_solution = Incumbent_solution
            print('Node number', Node_number)
            print('Pruned by integrality')
        
        else:
            # If feasible, Not Integer, Not better than Incumbent        
            if subproblem.getObjective().getValue() < Incumbent_obj_val:
                print('Node number', Node_number)
                print('Pruned by bound')
            
            # If feasible, Not Integer, But better than Incumbent  (?????????I have a question here (how to write def for creating submodels)????????)      
            if subproblem.getObjective().getValue() > Incumbent_obj_val:
            
                Incumbent_obj_val = subproblem.getObjective().getValue()
                Incumbent_solution = [var.X for var in subproblem.getVars()]
                print('Node number', Node_number)
                print('Continue branching')
            
                # Set subproblem 3
                subproblem3 = Model("subproblem3")
                x1 = subproblem3.addVar(lb=2, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="x1")
                x2 = subproblem3.addVar(lb=1, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="x2")
                subproblem3.setObjective(5.5 * x1 + 2.1 * x2, GRB.MAXIMIZE)
                subproblem3.addConstr(-1 * x1 + 1 * x2 <= 2, "c1")
                subproblem3.addConstr(8 * x1 + 2 * x2 <= 17, "c2")
    
                subproblems.append(subproblem3)
                
                
                # Set subproblem 4
                subproblem4 = Model("subproblem4")
                x1 = subproblem4.addVar(lb=2, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="x1")
                x2 = subproblem4.addVar(lb=0, ub=0, vtype=GRB.CONTINUOUS, name="x2")
                subproblem4.setObjective(5.5 * x1 + 2.1 * x2, GRB.MAXIMIZE)
                subproblem4.addConstr(-1 * x1 + 1 * x2 <= 2, "c1")
                subproblem4.addConstr(8 * x1 + 2 * x2 <= 17, "c2")
    
                subproblems.append(subproblem4)
        

    print('Best Objective is :', best_obj_val, 'Best integer solution is :', best_solution)
    return (best_obj_val, best_solution)
    
    

In [9]:
def solve_branch_and_bound():
    
    # Create the original model
    m = Model("branch_and_bound")
    x1 = m.addVar(lb=0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="x1")
    x2 = m.addVar(lb=0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="x2")
    m.setObjective(5.5 * x1 + 2.1 * x2, GRB.MAXIMIZE)
    m.addConstr(-1 * x1 + 1 * x2 <= 2, "c1")
    m.addConstr(8 * x1 + 2 * x2 <= 17, "c2")

    m.Params.TimeLimit = 5
    m.optimize()
    
    Node_number = 0
    first_obj_val = m.getAttr(GRB.Attr.ObjVal)
    first_solution = [var.x for var in m.getVars()]
    
    print('Node number', Node_number)
    print('first Objective Value is', first_obj_val)
    print('first solution is ', first_solution)
    
    
    subproblems = []
    
    # Set subproblem 1
    subproblem1 = Model("subproblem1")
    x1 = subproblem1.addVar(lb=0, ub=1, vtype=GRB.CONTINUOUS, name="x1")
    x2 = subproblem1.addVar(lb=0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="x2")
    subproblem1.setObjective(5.5 * x1 + 2.1 * x2, GRB.MAXIMIZE)
    subproblem1.addConstr(-1 * x1 + 1 * x2 <= 2, "c1")
    subproblem1.addConstr(8 * x1 + 2 * x2 <= 17, "c2")
    
    subproblems.append(subproblem1)
    
    # Set subproblem 2
    subproblem2 = Model("subproblem2")
    x1 = subproblem2.addVar(lb=2, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="x1")
    x2 = subproblem2.addVar(lb=0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="x2")
    subproblem2.setObjective(5.5 * x1 + 2.1 * x2, GRB.MAXIMIZE)
    subproblem2.addConstr(-1 * x1 + 1 * x2 <= 2, "c1")
    subproblem2.addConstr(8 * x1 + 2 * x2 <= 17, "c2")
    
    subproblems.append(subproblem2)
    
    branch_and_bound(m, [x1, x2], subproblems, Node_number)
    
if __name__ == "__main__":
    solve_branch_and_bound()
    h

Set parameter TimeLimit to value 5
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19045.2))

CPU model: 11th Gen Intel(R) Core(TM) i5-1145G7 @ 2.60GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0xe2091aba
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [2e+00, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 2e+01]
Presolve removed 2 rows and 2 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4080000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.408000000e+01
Node number 0
first Objective Value is 14.08
first solution is  [1.3, 3.3]
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 1

  Objective range  [2e+00, 6e+00]
  Bounds range     [2e+00, 2e+00]
  RHS range        [2e+00, 2e+01]
Presolve removed 2 rows and 2 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.2050000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.205000000e+01
Node number 2
Continue branching
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19045.2))

CPU model: 11th Gen Intel(R) Core(TM) i5-1145G7 @ 2.60GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0x9e43b255
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [2e+00, 6e+00]
  Bounds range     [1e+00, 2e+00]
  RHS range        [2e+00, 2e+01]
Presolve time: 0.01s

Solved in 0 iteratio